In [351]:
#Seperate the data for two obejcts
import pandas as pd

df = pd.read_csv("person_info_twopeople.csv")

threshold = 400

person1_data = {'FrameIndex': [], 'ClassName': [], 'X1': [], 'Y1': [], 'X2': [], 'Y2': [], 'CenterX': [], 'CenterY': [], 'Distance': [], 'World Coordinates': []}
person2_data = {'FrameIndex': [], 'ClassName': [], 'X1': [], 'Y1': [], 'X2': [], 'Y2': [], 'CenterX': [], 'CenterY': [], 'Distance': [], 'World Coordinates': []}

current_person = 1
prev_x = None

for index, row in df.iterrows():
    coordinates = eval(row['World Coordinates'])
    x = coordinates[0]
    
    if prev_x is not None and abs(x - prev_x) > threshold:
        current_person = 2 if current_person == 1 else 1
    
    if current_person == 1:
        for col in person1_data:
            person1_data[col].append(row[col])
    else:
        for col in person2_data:
            person2_data[col].append(row[col])
    
    prev_x = x

person1_df = pd.DataFrame(person1_data)
person2_df = pd.DataFrame(person2_data)

person1_df.to_csv('person1_data.csv', index=False)
person2_df.to_csv('person2_data.csv', index=False)


In [352]:
#Filter the data
import pandas as pd

df = pd.read_csv('person1_data.csv')

filtered_df = df[['FrameIndex', 'Distance']]

filtered_df.to_csv('camera_data1.csv', index=False)

In [353]:
import pandas as pd

df = pd.read_csv('person2_data.csv')

filtered_df = df[['FrameIndex', 'Distance']]

filtered_df.to_csv('camera_data2.csv', index=False)

In [354]:
#Use Kalman Filter
import numpy as np
import pandas as pd
from filterpy.kalman import KalmanFilter

df = pd.read_csv('camera_data1.csv')

x = df['FrameIndex'].values
y = df['Distance'].values

kf = KalmanFilter(dim_x=4, dim_z=2)
kf.F = np.array([[1, 0, 1, 0],
                 [0, 1, 0, 1],
                 [0, 0, 1, 0],
                 [0, 0, 0, 1]])
kf.H = np.array([[1, 0, 0, 0],
                 [0, 1, 0, 0]])
kf.R *= 0.01
kf.P *= 0.01
kf.Q *= 0.01

initial_state_mean = [x[0], y[0], 0, 0]
kf.x = initial_state_mean

filtered_states = []

for measurement in zip(x, y):
    kf.predict()
    kf.update(measurement)
    filtered_states.append(kf.x)

filtered_states = np.array(filtered_states)

with open('tracker_points1.txt', 'w') as f:
    for point in filtered_states:
        f.write(f"{point[0]}, {point[1]}\n")


In [355]:
import numpy as np
import pandas as pd
from filterpy.kalman import KalmanFilter

df = pd.read_csv('camera_data2.csv')

x = df['FrameIndex'].values
y = df['Distance'].values

kf = KalmanFilter(dim_x=4, dim_z=2)
kf.F = np.array([[1, 0, 1, 0],
                 [0, 1, 0, 1],
                 [0, 0, 1, 0],
                 [0, 0, 0, 1]])
kf.H = np.array([[1, 0, 0, 0],
                 [0, 1, 0, 0]])
kf.R *= 0.01
kf.P *= 0.01
kf.Q *= 0.01

initial_state_mean = [x[0], y[0], 0, 0]
kf.x = initial_state_mean

filtered_states = []

for measurement in zip(x, y):
    kf.predict()
    kf.update(measurement)
    filtered_states.append(kf.x)

filtered_states = np.array(filtered_states)

with open('tracker_points2.txt', 'w') as f:
    for point in filtered_states:
        f.write(f"{point[0]}, {point[1]}\n")


In [356]:
#Interpolation of the data
import numpy as np

data = np.loadtxt("tracker_points1.txt", delimiter=",")

x_values = data[:, 0]
y_values = data[:, 1]

new_x_values = np.linspace(min(x_values), max(x_values), num=475)

new_y_values = np.interp(new_x_values, x_values, y_values)

new_y_values_rounded = np.round(new_y_values, decimals=2) * 0.01

index_column = np.arange(1, 476)

new_data = np.column_stack((index_column, new_y_values_rounded))

headers = "index,distance"

np.savetxt("added_trackerpoints1.txt", new_data, delimiter=",", fmt="%.2f", header=headers, comments='')


In [357]:
import numpy as np

data = np.loadtxt("tracker_points2.txt", delimiter=",")

x_values = data[:, 0]
y_values = data[:, 1]

new_x_values = np.linspace(min(x_values), max(x_values), num=510)

new_y_values = np.interp(new_x_values, x_values, y_values)

new_y_values_rounded = np.round(new_y_values, decimals=2)*0.01

index_column = np.arange(1, 511)

new_data = np.column_stack((index_column, new_y_values_rounded))

headers = "index,distance"

np.savetxt("added_trackerpoints2.txt", new_data, delimiter=",", fmt="%.2f", header=headers, comments='')

In [358]:
#Angle calculation
import csv
import math

data = []
with open('person1_data.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data.append(row)

def calculate_angle(x_coord, distance):
    x_coord = int(x_coord)
    angle = math.degrees(math.atan(x_coord*0.1 / distance))
    return angle

output_data = []
for entry in data:
    world_coords = eval(entry['World Coordinates'])
    angle = calculate_angle(world_coords[0], float(entry['Distance']))
    output_data.append({'FrameIndex': entry['FrameIndex'], 'Angle': angle})

output_file = 'angles1.csv'
with open(output_file, 'w', newline='') as csvfile:
    fieldnames = ['FrameIndex', 'Angle']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    for entry in output_data:
        writer.writerow(entry)



In [359]:
import csv
import math

data = []
with open('person2_data.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data.append(row)

def calculate_angle(x_coord, distance):
    x_coord = int(x_coord)
    angle = math.degrees(math.atan(x_coord*0.1 / distance))
    return angle

output_data = []
for entry in data:
    world_coords = eval(entry['World Coordinates'])
    angle = calculate_angle(world_coords[0], float(entry['Distance']))
    output_data.append({'FrameIndex': entry['FrameIndex'], 'Angle': angle})

output_file = 'angles2.csv'
with open(output_file, 'w', newline='') as csvfile:
    fieldnames = ['FrameIndex', 'Angle']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    for entry in output_data:
        writer.writerow(entry)



In [360]:
import numpy as np
import pandas as pd
from filterpy.kalman import KalmanFilter

df = pd.read_csv('angles1.csv')

x = df['FrameIndex'].values
y = df['Angle'].values

kf = KalmanFilter(dim_x=4, dim_z=2)
kf.F = np.array([[1, 0, 1, 0],
                 [0, 1, 0, 1],
                 [0, 0, 1, 0],
                 [0, 0, 0, 1]])
kf.H = np.array([[1, 0, 0, 0],
                 [0, 1, 0, 0]])
kf.R *= 0.01
kf.P *= 0.01
kf.Q *= 0.01

initial_state_mean = [x[0], y[0], 0, 0]
kf.x = initial_state_mean

filtered_states = []

for measurement in zip(x, y):
    kf.predict()
    kf.update(measurement)
    filtered_states.append(kf.x)

filtered_states = np.array(filtered_states)

with open('angle_tracker1.txt', 'w') as f:
    for point in filtered_states:
        f.write(f"{point[0]}, {point[1]}\n")



In [361]:
import numpy as np
import pandas as pd
from filterpy.kalman import KalmanFilter

df = pd.read_csv('angles2.csv')

x = df['FrameIndex'].values
y = df['Angle'].values

kf = KalmanFilter(dim_x=4, dim_z=2)
kf.F = np.array([[1, 0, 1, 0],
                 [0, 1, 0, 1],
                 [0, 0, 1, 0],
                 [0, 0, 0, 1]])
kf.H = np.array([[1, 0, 0, 0],
                 [0, 1, 0, 0]])
kf.R *= 0.01
kf.P *= 0.01
kf.Q *= 0.01

initial_state_mean = [x[0], y[0], 0, 0]
kf.x = initial_state_mean

filtered_states = []

for measurement in zip(x, y):
    kf.predict()
    kf.update(measurement)
    filtered_states.append(kf.x)

filtered_states = np.array(filtered_states)

with open('angle_tracker2.txt', 'w') as f:
    for point in filtered_states:
        f.write(f"{point[0]}, {point[1]}\n")



In [362]:
import numpy as np

data = np.loadtxt("angle_tracker1.txt", delimiter=",")

x_values = data[:, 0]
y_values = data[:, 1]

new_x_values = np.linspace(min(x_values), max(x_values), num=475)

new_y_values = np.interp(new_x_values, x_values, y_values)

new_y_values_rounded = np.round(new_y_values, decimals=2)

index_column = np.arange(1, 476)

new_data = np.column_stack((index_column, new_y_values_rounded))

headers = "index,angle"

np.savetxt("added_angle1.txt", new_data, delimiter=",", fmt="%.2f", header=headers, comments='')

In [363]:
import numpy as np

data = np.loadtxt("angle_tracker2.txt", delimiter=",")

x_values = data[:, 0]
y_values = data[:, 1]

new_x_values = np.linspace(min(x_values), max(x_values), num=510)

new_y_values = np.interp(new_x_values, x_values, y_values)

new_y_values_rounded = np.round(new_y_values, decimals=2)

index_column = np.arange(1, 511)

new_data = np.column_stack((index_column, new_y_values_rounded))

headers = "index,angle"

np.savetxt("added_angle2.txt", new_data, delimiter=",", fmt="%.2f", header=headers, comments='')

In [364]:
#Velocity calculation
import pandas as pd
import numpy as np

df = pd.read_csv("person1_data.csv")

velocities = []

for i in range(1, len(df)):

    current_coordinates = eval(df.iloc[i]["World Coordinates"])
    previous_coordinates = eval(df.iloc[i-1]["World Coordinates"])
    
    current_x, current_y, current_z = current_coordinates
    previous_x, previous_y, previous_z = previous_coordinates
    
    delta_x = current_x - previous_x
    delta_y = current_y - previous_y
    delta_z = current_z - previous_z

    print('x=', delta_x)
    print('z=',delta_z)
    
    distance = np.sqrt((0.001*delta_x)**2 + (0.001*delta_z)**2)

    print('distance=', distance)
    
    velocity = distance / 0.5
    
    velocities.append(velocity)

velocities = [np.nan] + velocities

df["Velocity"] = velocities

df["Velocity"].fillna(0, inplace=True)

df[['FrameIndex', 'Velocity']].to_csv('velocity1.csv', index=False)


x= -137.55813598632812
z= 354.0
distance= 0.37978709927541393
x= -139.031494140625
z= 302.0
distance= 0.33246617326124267
x= -124.062255859375
z= 323.0
distance= 0.3460064209359662
x= -6.27239990234375
z= 231.0
distance= 0.23108514231887547
x= -70.4305419921875
z= 290.0
distance= 0.2984299938768107
x= 22.02813720703125
z= 379.0
distance= 0.37963961704333726
x= -25.4429931640625
z= 252.0
distance= 0.25328115978324667
x= -4.73187255859375
z= 313.0
distance= 0.31303576571681196
x= 18.75067138671875
z= 371.0
distance= 0.3714735356353837
x= -58.99530029296875
z= 300.0
distance= 0.30574572025893926
x= 7.1549072265625
z= 621.0
distance= 0.6210412165850353
x= 39.3433837890625
z= 197.0
distance= 0.2008902731542109
x= -20.399169921875
z= 457.0
distance= 0.45745505367576994
x= 26.2105712890625
z= 554.0
distance= 0.554619684150589
x= -110.974609375
z= 529.0
distance= 0.5405149062939281
x= 100.59539794921875
z= -150.0
distance= 0.1806085105651494
x= -89.86700439453125
z= 649.0
distance= 0.655192398

C:\Users\19799\AppData\Local\Temp\ipykernel_28024\1182118741.py:35: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Velocity"].fillna(0, inplace=True)


In [365]:
import pandas as pd
import numpy as np

df = pd.read_csv("person2_data.csv")

velocities = []

for i in range(1, len(df)):

    current_coordinates = eval(df.iloc[i]["World Coordinates"])
    previous_coordinates = eval(df.iloc[i-1]["World Coordinates"])
    
    current_x, current_y, current_z = current_coordinates
    previous_x, previous_y, previous_z = previous_coordinates
    
    delta_x = current_x - previous_x
    delta_y = current_y - previous_y
    delta_z = current_z - previous_z

    print('x=', delta_x)
    print('z=',delta_z)
    
    distance = np.sqrt((0.001*delta_x)**2 + (0.001*delta_z)**2)

    print('distance=', distance)
    
    velocity = distance / 0.5
    
    velocities.append(velocity)

velocities = [np.nan] + velocities

df["Velocity"] = velocities

df["Velocity"].fillna(0, inplace=True)

df[['FrameIndex', 'Velocity']].to_csv('velocity2.csv', index=False)


x= 123.70816040039062
z= 316.0
distance= 0.3393518954561014
x= -21.419586181640625
z= 208.0
distance= 0.2090999729129412
x= 111.41354370117188
z= 421.0
distance= 0.4354927987005674
x= 2.85736083984375
z= 216.0
distance= 0.21601889850420278
x= -48.973388671875
z= 296.0
distance= 0.3000239870377142
x= 65.97637939453125
z= 423.0
distance= 0.42811433360495077
x= -70.00115966796875
z= 334.0
distance= 0.3412567396475277
x= 3.448974609375
z= 386.0
distance= 0.3860154082751829
x= 49.62615966796875
z= 316.0
distance= 0.3198730306283897
x= -72.51376342773438
z= 269.0
distance= 0.27860230775507483
x= 77.56838989257812
z= 649.0
distance= 0.6536190443297434
x= 24.7205810546875
z= 598.0
distance= 0.5985107410295001
x= -85.68511962890625
z= 103.0
distance= 0.1339811170494558
x= 54.20654296875
z= 307.0
distance= 0.3117488561336239
x= 17.2977294921875
z= 1275.0
distance= 1.2751173324230147
x= -91.0743408203125
z= -757.0
distance= 0.7624588746652862
x= 57.466552734375
z= 712.0
distance= 0.71431533980670

C:\Users\19799\AppData\Local\Temp\ipykernel_28024\3112448462.py:35: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Velocity"].fillna(0, inplace=True)


In [366]:
import numpy as np
import pandas as pd
from filterpy.kalman import KalmanFilter

df = pd.read_csv('velocity1.csv')

x = df['FrameIndex'].values
y = df['Velocity'].values

kf = KalmanFilter(dim_x=4, dim_z=2)
kf.F = np.array([[1, 0, 1, 0],
                 [0, 1, 0, 1],
                 [0, 0, 1, 0],
                 [0, 0, 0, 1]])
kf.H = np.array([[1, 0, 0, 0],
                 [0, 1, 0, 0]])
kf.R *= 0.01
kf.P *= 0.01
kf.Q *= 0.01

initial_state_mean = [x[0], y[0], 0, 0]
kf.x = initial_state_mean

filtered_states = []

for measurement in zip(x, y):
    kf.predict()
    kf.update(measurement)
    filtered_states.append(kf.x)

filtered_states = np.array(filtered_states)

with open('velocity_tracker1.txt', 'w') as f:
    for point in filtered_states:
        f.write(f"{point[0]}, {point[1]}\n")



In [367]:
import numpy as np
import pandas as pd
from filterpy.kalman import KalmanFilter

df = pd.read_csv('velocity2.csv')

x = df['FrameIndex'].values
y = df['Velocity'].values

kf = KalmanFilter(dim_x=4, dim_z=2)
kf.F = np.array([[1, 0, 1, 0],
                 [0, 1, 0, 1],
                 [0, 0, 1, 0],
                 [0, 0, 0, 1]])
kf.H = np.array([[1, 0, 0, 0],
                 [0, 1, 0, 0]])
kf.R *= 0.01
kf.P *= 0.01
kf.Q *= 0.01

initial_state_mean = [x[0], y[0], 0, 0]
kf.x = initial_state_mean

filtered_states = []

for measurement in zip(x, y):
    kf.predict()
    kf.update(measurement)
    filtered_states.append(kf.x)

filtered_states = np.array(filtered_states)

with open('velocity_tracker2.txt', 'w') as f:
    for point in filtered_states:
        f.write(f"{point[0]}, {point[1]}\n")



In [368]:
import numpy as np

data = np.loadtxt("velocity_tracker1.txt", delimiter=",")

x_values = data[:, 0]
y_values = data[:, 1]

new_x_values = np.linspace(min(x_values), max(x_values), num=475)

new_y_values = np.interp(new_x_values, x_values, y_values)

new_y_values_rounded = np.round(new_y_values, decimals=2)

index_column = np.arange(1, 476)

new_data = np.column_stack((index_column, new_y_values_rounded))

headers = "index,velocity"

np.savetxt("added_velocity1.txt", new_data, delimiter=",", fmt="%.2f", header=headers, comments='')

In [369]:
import numpy as np

data = np.loadtxt("velocity_tracker2.txt", delimiter=",")

x_values = data[:, 0]
y_values = data[:, 1]

new_x_values = np.linspace(min(x_values), max(x_values), num=510)

new_y_values = np.interp(new_x_values, x_values, y_values)

new_y_values_rounded = np.round(new_y_values, decimals=2)

index_column = np.arange(1, 511)

new_data = np.column_stack((index_column, new_y_values_rounded))

headers = "index,velocity"

np.savetxt("added_velocity2.txt", new_data, delimiter=",", fmt="%.2f", header=headers, comments='')